<a href="https://colab.research.google.com/github/susumu2357/US-patent-analysis/blob/master/colab/Dataset_with_IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!gsutil cp gs://mlstudy-phys/data/citations_info_3000+3000.df.gz ./citations_info_3000+3000.df.gz
!gsutil cp gs://mlstudy-phys/data/grants_for_3000+3000.df.gz ./grants_for_3000+3000.df.gz
!gsutil cp gs://mlstudy-phys/data/testset_app_3000.df.gz ./testset_app_3000.df.gz
!gsutil cp gs://mlstudy-phys/data/training_app_3000.df.gz ./training_app_3000.df.gz 

Copying gs://mlstudy-phys/data/citations_info_3000+3000.df.gz...
/ [1 files][506.5 KiB/506.5 KiB]                                                
Operation completed over 1 objects/506.5 KiB.                                    
Copying gs://mlstudy-phys/data/grants_for_3000+3000.df.gz...
\ [1 files][129.4 MiB/129.4 MiB]                                                
Operation completed over 1 objects/129.4 MiB.                                    
Copying gs://mlstudy-phys/data/testset_app_3000.df.gz...
\
Operation completed over 1 objects/45.5 MiB.                                     
Copying gs://mlstudy-phys/data/training_app_3000.df.gz...
- [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     


In [0]:
import pandas as pd

citations_info = pd.read_pickle("./citations_info_3000+3000.df.gz")
grants = pd.read_pickle("./grants_for_3000+3000.df.gz")
testset_app_3000 = pd.read_pickle("./testset_app_3000.df.gz")
training_app_3000 = pd.read_pickle("./training_app_3000.df.gz")

In [0]:
training_app_3000["xml"][3]

'<us-patent-application lang="EN" dtd-version="v4.3 2012-12-04" file="US20140260920A1-20140918.XML" status="PRODUCTION" id="us-patent-application" country="US" date-produced="20140830" date-publ="20140918">\n<us-bibliographic-data-application lang="EN" country="US">\n<publication-reference>\n<document-id>\n<country>US</country>\n<doc-number>20140260920</doc-number>\n<kind>A1</kind>\n<date>20140918</date>\n</document-id>\n</publication-reference>\n<application-reference appl-type="utility">\n<document-id>\n<country>US</country>\n<doc-number>14204411</doc-number>\n<date>20140311</date>\n</document-id>\n</application-reference>\n<us-application-series-code>14</us-application-series-code>\n<priority-claims>\n<priority-claim sequence="01" kind="national">\n<country>JP</country>\n<doc-number>2013-049035</doc-number>\n<date>20130312</date>\n</priority-claim>\n<priority-claim sequence="02" kind="national">\n<country>JP</country>\n<doc-number>2014-047225</doc-number>\n<date>20140311</date>\n</p

In [0]:
import re

def ipc_section(whole):
  list = re.findall(r'<section>(.)</section>', whole)
  return list

In [0]:
result = training_app_3000["xml"][3:4].map(ipc_section).values[0]
result

['G', 'G', 'G', 'G']

In [0]:
training_app_section =  training_app_3000["xml"].map(ipc_section)
training_app_section.head()

0                                        [A]
1                                        [G]
2                                        [G]
3                               [G, G, G, G]
4    [A, A, A, A, A, C, C, C, C, C, A, A, A]
Name: xml, dtype: object

In [0]:
import numpy as np

In [0]:
section_to_vec_dict = {
    "A":[0,0,0,0,0,0,0,1],
    "B":[0,0,0,0,0,0,1,0],
    "C":[0,0,0,0,0,1,0,0],
    "D":[0,0,0,0,1,0,0,0],
    "E":[0,0,0,1,0,0,0,0],
    "F":[0,0,1,0,0,0,0,0],
    "G":[0,1,0,0,0,0,0,0],
    "H":[1,0,0,0,0,0,0,0],    
}

def section_to_vec(list):
  vec = np.zeros(8)
  for section in list:
    if section not in section_to_vec_dict.keys():
      vec = vec + np.zeros(8)
    else:
      vec = vec + np.array(section_to_vec_dict[section])
  return vec/np.linalg.norm(vec) if np.linalg.norm(vec) >0 else vec

In [0]:
training_app_section[0:5].map(section_to_vec)

0             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
1             [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2             [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3             [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.52999894000318, 0....
Name: xml, dtype: object

In [0]:
training_app_section_vec = training_app_section.map(section_to_vec)
training_app_section_vec.shape

(3000,)

In [0]:
section_to_vec_dict.keys()

dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

In [0]:
test_app_section =  testset_app_3000["xml"].map(ipc_section)
test_app_section.head()

0                         [G]
1             [A, A, A, C, C]
2                      [B, A]
3    [C, C, A, A, C, A, C, A]
4                         [H]
Name: xml, dtype: object

In [0]:
test_app_section_vec = test_app_section.map(section_to_vec)
test_app_section_vec.shape

(3000,)

In [0]:
test_app_section_vec[0:10]

0             [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.5547001962252291, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...
4             [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
5             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
6    [0.7071067811865475, 0.7071067811865475, 0.0, ...
7             [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
8    [0.5773502691896258, 0.5773502691896258, 0.577...
9             [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Name: xml, dtype: object

In [0]:
grants["xml"][6410]

'<us-patent-grant lang="EN" dtd-version="v4.2 2006-08-23" file="US08328055-20121211.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20121127" date-publ="20121211">\n<us-bibliographic-data-grant>\n<publication-reference>\n<document-id>\n<country>US</country>\n<doc-number>08328055</doc-number>\n<kind>B1</kind>\n<date>20121211</date>\n</document-id>\n</publication-reference>\n<application-reference appl-type="utility">\n<document-id>\n<country>US</country>\n<doc-number>12984712</doc-number>\n<date>20110105</date>\n</document-id>\n</application-reference>\n<us-application-series-code>12</us-application-series-code>\n<us-term-of-grant>\n<us-term-extension>210</us-term-extension>\n</us-term-of-grant>\n<classifications-ipcr>\n<classification-ipcr>\n<ipc-version-indicator><date>20060101</date></ipc-version-indicator>\n<classification-level>A</classification-level>\n<section>A</section>\n<class>45</class>\n<subclass>F</subclass>\n<main-group>3</main-group>\n<subgroup>1

In [0]:
import re

def ipc_section_grant(whole):
  list = re.findall(r'<main-classification>([A-H]).{6,9}</main-classification>', whole)
  return list

In [0]:
grant_section_1 =  grants["xml"].map(ipc_section_grant)
grant_section_2 =  grants["xml"].map(ipc_section)
grant_section = [section_1+section_2 for section_1, section_2 in zip(grant_section_1, grant_section_2)]
grant_section = pd.Series(grant_section)
grant_section.shape, grant_section_1.shape,grant_section_2.shape

((6440,), (6440,), (6440,))

In [0]:
grant_section[50:100]

50                                     [H]
51                                      []
52                                     [G]
53                                     [G]
54                                     [G]
55                                     [G]
56                                     [G]
57                                     [G]
58                                  [G, G]
59                                  [A, D]
60                                     [B]
61                                     [F]
62                                     [F]
63                                     [F]
64                                     [G]
65                                     [A]
66                                     [B]
67                                     [A]
68                      [A, C, A, A, C, A]
69                                     [G]
70                                     [H]
71                                     [F]
72                                     [G]
73         

In [0]:
grant_section_vec = grant_section.map(section_to_vec)
grant_section_vec.shape

(6440,)

In [0]:
grant_section_vec[51]

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
grant_section_vec[59]

array([0.        , 0.        , 0.        , 0.        , 0.70710678,
       0.        , 0.        , 0.70710678])

In [0]:
import gzip
import pickle

filename = "train_ipc_section.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(training_app_section_vec, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://train_ipc_section.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][ 49.1 KiB/ 49.1 KiB]                                                
Operation completed over 1 objects/49.1 KiB.                                     


In [0]:
import gzip
import pickle

filename = "test_ipc_section.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(test_app_section_vec, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://test_ipc_section.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][ 49.2 KiB/ 49.2 KiB]                                                
Operation completed over 1 objects/49.2 KiB.                                     


In [0]:
import gzip
import pickle

filename = "grant_ipc_section.pkl.gz"

with gzip.open(filename, 'w') as f:
  pickle.dump(grant_section_vec, f)
!gsutil cp {filename} gs://mlstudy-phys/data/

Copying file://grant_ipc_section.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][101.3 KiB/101.3 KiB]                                                
Operation completed over 1 objects/101.3 KiB.                                    
